In [17]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from tqdm import tqdm 

In [18]:
df = pd.read_csv('data/TweetsNew.csv').drop('Unnamed: 0', axis =1)
df.dropna(subset = ['Text', 'Emotion_New'], inplace = True)
df.head()

,Text,Item,Emotion,Emotion_New
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion,0.0
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion,2.0
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion,2.0
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion,0.0
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion,2.0


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8936 entries, 0 to 9092
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Text         8936 non-null   object 
 1   Item         3282 non-null   object 
 2   Emotion      8936 non-null   object 
 3   Emotion_New  8936 non-null   float64
dtypes: float64(1), object(3)
memory usage: 349.1+ KB


In [20]:
import nltk 
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import RegexpTokenizer, sent_tokenize, word_tokenize, TweetTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [22]:
import enchant
from pprint import pprint

en_us = enchant.Dict("en_US")

phrases = df.Text.values


for i, phrase in enumerate(df.Text):
    phrases[i] = ' '.join(w for w in phrase.split() if en_us.check(w))

df.Text = phrases

In [23]:
word_tokenizer = RegexpTokenizer(r'\w+')
tweet_token = TweetTokenizer()
df.Text = df.Text.map(lambda x: tweet_token.tokenize(x.lower()))
df.Text = df.Text.map(lambda x: ' '.join(x))
df.Text= df.Text.map(lambda x: word_tokenizer.tokenize(x.lower()))
df.Text = df.Text.map(lambda x: ' '.join(x))

In [24]:
x_train, x_test, y_train, y_test = train_test_split(df.Text, df.Emotion_New, stratify = df.Emotion_New, train_size = .75, 
                                                   random_state = 10)

In [25]:
x_train 

1006             grown men playing google party speakeasy
8275    the secret search sauce google and spill the b...
8081    4 finding an in the back of austin yellow drop...
5779    rt giving added value to location based servic...
1679    hoping a or popup store opens up so i can get ...
                              ...                        
8709    within hours of buying an glenda watson cerebr...
6120    rt make sure you re donating to the japanese r...
5476                      rt apple to open pop up shop at
881     purchased mexican mavis by boy bear on first l...
3771      rt going to at check in while in location based
Name: Text, Length: 6702, dtype: object

In [26]:
stop = stopwords.words('english')

vectorizer = CountVectorizer(stop_words = stop, max_features = 5000, ngram_range=(1,3))
clean_train = x_train.values
clean_test = x_test.values

train_features =vectorizer.fit_transform(clean_train).toarray()
test_features = vectorizer.fit_transform(clean_test).toarray()

In [31]:
for i in train_features[0]: 
    if i != 0: 
        print(i)

1
1
1
1
1
1


In [27]:
import pickle
#unsampled 

pickle.dump(train_features, open('../Pickles/x_train.p', 'wb'))
pickle.dump(test_features, open('../Pickles/x_test.p', 'wb'))
pickle.dump(y_train, open('../Pickles/y_train.p', 'wb'))
pickle.dump(y_test, open('../Pickles/y_test.p', 'wb'))

In [28]:
print(train_features.shape, test_features.shape)
print(y_train.shape, y_test.shape)

(6702, 5000) (2234, 5000)
(6702,) (2234,)
